# Desafio de Análise de Dados

## Cenário

Apresentar informações sobre o estado de um município, com relação ao COVID-19, a um gestor, de forma que ele tenha informações suficientes para tomar decisões estratégicas. Para isso serão utilizados dados de indicadores já presentes no [FarolCovid](https://farolcovid.coronacidades.org/) (incluindo o módulo `Saúde em Ordem`), além de outros indicadores que serão calculados aqui.

## Possíveis decisões da gestão

Esta análise será direcionada pelas principais possíveis decisões que um gestor de um município tem que tomar relacionadas ao tema. Aqui serão consideradas:

1. Imposição de confinamento
2. Aquisição de testes
3. Rigidez no controle de entrada de pessoas no munícipio
4. Abertura de Comércio

Claro que outras medidas podem ser tomadas além destas, como: montagens de mais leitos, solicitação de ajuda externa, doação de alimentos, redistribuição de pacientes entre hospitais, etc. Mas estas últimas não serão diretamente referenciandas aqui.

## Visão Geral do Município

### Comparação com municípios próximos

In [1]:
import pandas as pd


df = pd.read_csv('http://datasource.coronacidades.org/br/cities/cases/full')

In [2]:
import pandas as pd
from geopy.distance import distance


df_states = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/estados.csv')
stateabbr_to_code = df_states.set_index('codigo_uf')['uf'].to_dict()

df_geo = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/municipios.csv')
df_geo['uf'] = df_geo['codigo_uf'].map(stateabbr_to_code)
df_geo.pop('codigo_uf')

df_geo.set_index(['uf', 'nome'], inplace=True)
df_geo.sort_index(inplace=True)


# Improve speed with dict with surround states and memoization
def get_distances(state_abbr, city_name):
    city = df_geo.loc[state_abbr, city_name]
    city_coords = city[['latitude', 'longitude']]

    def get_distance(row):
        this_coords = row[['latitude', 'longitude']]
        return distance(city_coords, this_coords).km
        
    distances = df_geo.apply(get_distance, axis='columns')
    distances.sort_values(inplace=True)
    
    return distances.rename('distance')


def get_nearby_cities(state_abbr, city_name, radius_km):
    distances = get_distances(state_abbr, city_name)
    distances = distances[distances <= radius_km]
    
    return distances.index

In [3]:
state, city = 'SC', 'Joinville'
radius_km = 100

nearby_cities = get_nearby_cities(state, city, radius_km)
nearby_cities = [f'{city} - {uf}' for uf, city in nearby_cities]

nearby_cities

['Joinville - SC',
 'Araquari - SC',
 'São Francisco do Sul - SC',
 'Guaramirim - SC',
 'Schroeder - SC',
 'Balneário Barra do Sul - SC',
 'Jaraguá do Sul - SC',
 'Garuva - SC',
 'Itapoá - SC',
 'São João do Itaperiú - SC',
 'Massaranduba - SC',
 'Barra Velha - SC',
 'Corupá - SC',
 'Campo Alegre - SC',
 'Luiz Alves - SC',
 'São Bento do Sul - SC',
 'Balneário Piçarras - SC',
 'Tijucas do Sul - PR',
 'Guaratuba - PR',
 'Penha - SC',
 'Pomerode - SC',
 'Agudos do Sul - PR',
 'Matinhos - PR',
 'Piên - PR',
 'Rio dos Cedros - SC',
 'Ilhota - SC',
 'Rio Negrinho - SC',
 'Navegantes - SC',
 'Itajaí - SC',
 'Gaspar - SC',
 'Blumenau - SC',
 'Timbó - SC',
 'Benedito Novo - SC',
 'Mandirituba - PR',
 'Indaial - SC',
 'Pontal do Paraná - PR',
 'Doutor Pedrinho - SC',
 'Balneário Camboriú - SC',
 'Quitandinha - PR',
 'Camboriú - SC',
 'Fazenda Rio Grande - PR',
 'Rodeio - SC',
 'Guabiruba - SC',
 'Brusque - SC',
 'Ascurra - SC',
 'Itapema - SC',
 'Campo do Tenente - PR',
 'Morretes - PR',
 'São 